In [1]:
!pip install torch
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import PIL

from torch.autograd import Variable

# check the system we're on, gpu
use_cuda = torch.cuda.is_available()

np.load.__defaults__=(None, True, True, 'ASCII')

^C


ModuleNotFoundError: No module named 'torch'

In [ ]:
class HyperParameters():
    def __init__(self):
        self.data = '../Datasets/sketchrnn_chair.npz'
        self.encoder_hidden_size = 64
        self.decoder_hidden_size = 128
        # latent vector
        self.Nz = 32
        
        #return
        self.M = 20
        self.dropout = 0.9
        self.batch_size = 100
        self.eta_min = 0.01
        self.R = 0.99995
        self.KL_min = 0.2
        self.wKL = 0.5
        self.lr = 0.001
        self.lr_decay = 0.9999
        self.min_lr = 0.00001
        self.grad_clip = 1.
        self.temperature = 0.4
        self.max_seq_length = 200

hp = HyperParameters()

In [ ]:
class DataLoader():
    def __init__(self, file):
        self.file = file
        
    def load_npz(self):
        data = np.load(file=self.file, encoding='latin1', allow_pickle=True)
        return data

dataLoader = DataLoader(hp.data)
data = dataLoader.load_npz()

In [ ]:
class DataHandler:
    def __init__(self, data):
        self.data = data
    
    def padding(self, strokedata, max_len):
        s = torch.from_numpy(strokedata)
        s_len = s.shape[0]
        result = torch.zeros((max_len, 5))
        result[0:s_len, 0:2] = s[:, 0:2]
        result[0:s_len, 3] = s[:, 2]
        result[0:s_len, 2] = 1 - result[0:s_len, 3]    # 1 to 0, 0 to 1
        if s_len < max_len:
            result[(s_len - 1):, 4] = 1
        return result
        
    def handle(self):
        max_len = 0    # max length of a sketch - max number of stroke vectors in a sketch
        for datatype in self.data:
            for strokedata in self.data[datatype]:
                if strokedata.shape[0] > max_len:
                    max_len = strokedata.shape[0]
        # lengths of data - the number of sketchs in data
        train_len = self.data['train'].shape[0]
        test_len = self.data['test'].shape[0]
        valid_len = self.data['valid'].shape[0]
        
        data_train = torch.zeros((train_len, max_len, 5), dtype=float)
        data_test = torch.zeros((test_len, max_len, 5), dtype=float)
        data_valid = torch.zeros((valid_len, max_len, 5), dtype=float)
        
        i = j = k = 0
        for datatype in self.data:
            for strokedata in self.data[datatype]:
                if datatype == 'train':
                    data_train[i] = self.padding(strokedata, max_len)
                    i += 1
                if datatype == 'test':
                    data_test[j] = self.padding(strokedata, max_len)
                    j += 1
                if datatype == 'valid':
                    data_valid[k] = self.padding(strokedata, max_len)
                    k += 1 

        return data_train, data_test, data_valid
    
        
datahandler = DataHandler(data)
data_train, data_test, data_valid = datahandler.handle()  

In [ ]:
# class Encoder(nn.Module):
#     def __init__(self):
#         super().__init__()
#         # bidirectional lstm:
#         self.lstm = nn.LSTM(5, hp.encoder_hidden_size, bidirectional=True)
#         # create mu and sigma from lstm's last output:
#         self.fc_mu = nn.Linear(2*hp.encoder_hidden_size, hp.Nz)
#         self.fc_sigma = nn.Linear(2*hp.encoder_hidden_size, hp.Nz)
#         # active dropout:
#         # self.train()

#     def forward(self, inputs, batch_size, hidden_cell=None):
#         if hidden_cell is None:
#             # then must init with zeros
#             if use_cuda:
#                 hidden = torch.zeros(2, batch_size, hp.encoder_hidden_size).cuda()
#                 cell = torch.zeros(2, batch_size, hp.encoder_hidden_size).cuda()
#             else:
#                 hidden = torch.zeros(2, batch_size, hp.encoder_hidden_size)
#                 cell = torch.zeros(2, batch_size, hp.encoder_hidden_size)
#             hidden_cell = (hidden, cell)
#         _, (hidden,cell) = self.lstm(inputs.float(), hidden_cell)
#         # hidden is (2, batch_size, hidden_size), we want (batch_size, 2*hidden_size):
#         hidden_forward, hidden_backward = torch.split(hidden,1,0)
#         hidden_cat = torch.cat([hidden_forward.squeeze(0), hidden_backward.squeeze(0)],1)
#         # mu and sigma:
#         mu = self.fc_mu(hidden_cat)
#         sigma_hat = self.fc_sigma(hidden_cat)
#         sigma = torch.exp(sigma_hat/2.)
#         # N ~ N(0,1)
#         z_size = mu.size()
                                   
#         if use_cuda:
#             N = torch.normal(torch.zeros(z_size),torch.ones(z_size)).cuda()
#         else:
#             N = torch.normal(torch.zeros(z_size),torch.ones(z_size))
#         z = mu + sigma*N
#         # mu and sigma_hat are needed for LKL loss
#         return z, mu, sigma_hat
    
# encoder = Encoder()
# encoder.train()
# z, _, _ = encoder(data_train, hp.batch_size)

In [ ]:
# a = torch.rand((3,1,5))
b = torch.tensor([[1,2], 
                  [3,4]])
c = torch.tensor([[5,6],
                  [7,8]])
d = torch.stack([b.unsqueeze(0),c.unsqueeze(0)], 0)
e = (b.unsqueeze(0).contiguous(),c.unsqueeze(0).contiguous())
print(d.is_contiguous())
print(d)
print(e)
# print(a.squeeze(0).shape)

In [ ]:
class Decoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(hp.Nz + 5, hp.decoder_hidden_size)
        self.fc_hc = nn.Linear(hp.Nz, 2*hp.decoder_hidden_size)
        
    def forward(self, inputs, z, hidden_cell=None):
        if hidden_cell is None:       
            hidden, cell = torch.split(F.tanh(self.fc_hc(z)), hp.decoder_hidden_size, 1)
            hidden_cell = (hidden.unsqueeze(0), cell.unsqueeze(0))
            

In [ ]:
"""
This for after training, the paper says we can sample sketches for the model.
During the sampling process we generate parameters for both GMM and categorical
distributions at each time step, and sample an outcome S'i for that time step.

GMM ->
Categorical distributions -> calculated using outputs as logit values

We continue to sample until p3, so the end of sketch, is 1 or the number of iterations
has reached Nmax (length of longest sketch in training dataset). 
Sampling the output is not deterministic, it's a random sequence,
conditioned on the input latent vector z. We can control the level of randomness we'd like
our samples to have during the sampling process by using a temperature parameter.

- state_q_hat tends to state_q_hat / temperature
- prime_k tends to prime_k / temperature
- sigma_squared_x tends to sigma_squared_temperature_x
- sigma_squared_y tends to sigma_squared_temperature_y

- the softmax parameters can be scaled, of the categorical distribution and also sigma
parameters of the bivariate normal distribution by a temperature parameter temperature.
To control the level of randomness in our samples.
- temperature is usually between 0 and 1, so limit T -> 0, model becomes deterministic and
samples will consist of the most likely point in the probability density function.

- Unconditinal generation uses this temperature methodology to control randomness of the
distribution

Unconditional generation -> train model to generate sketches unconditionally where train
decoder RNN module
-> Initial hidden states and cell states of decoder RNN are initialized to zero, inputs xi
of decoder RNN at each step is only Si-1 or Si-1', don't need to concatenate latent vectorZ

"""

In [ ]:
# This is part of the model

def tweak_temperature(self, pi_pdf):
    # sigma x = exp(sigma_hat), sigma_hat = log(Sigma x)
    pi_pdf = np.log(pi_pdf)/hp.temperature
    pi_pdf -= pi_pdf.max()
    pi_pdf = np.exp(pi_pdf)
    pi_pdf /= pi_pdf.sum()
    return pi_pdf

def sample_sketch_state(self):
    # Get GMM index for mixture distribution
    pi = self.pi.data[0,0,:].numpy()
    pi = tweak_temperature(pi)
    pi_idx = np.random.choice(hp.M, p=pi)
    
    # Get the pen state, or sketch state
    pi = self.q.data[0,0,:].numpy()
    q = tweak_temperature(q)
    q_idx = np.random.choice(3, p=q)
    
    # Get mixture parameters
    mu_x = self.mu_x.data[0,0,pi_idx]
    mu_y = self.mu_y.data[0,0,pi_idx]
    sigma_x = self.sigma_x.data[0,0,pi_idx]
    sigma_y = self.sigma_y.data[0,0,pi_idx]
    rho_xy = self.rho_xy.data[0,0,pi_idx]
    x,y = sample_bivariate_normal(mu_x,mu_y,sigma_x,sigma_y,rho_xy,greedy=False)
    next_state = torch.zeros(5)
    next_state[0] = x
    next_state[1] = y
    next_state[q_idx+2] = 1
    return Variable(next_state).view(1,1,-1),x,y,q_idx==1,q_idx==2

In [ ]:
class Model():
    def __init__(self):
        if use_cuda:
            self.encoder = EncoderRNN().cuda()
            self.decoder = DecoderRNN().cuda()
        else:
            self.encoder = EncoderRNN()
            self.decoder = DecoderRNN()
        #self.encoder_optimizer = optim.Adam(self.encoder.parameters(), hp.lr)
        #self.decoder_optimizer = optim.Adam(self.decoder.parameters(), hp.lr)
        #self.eta_step = hp.eta_min
    
# take the parameters to form the normal distribution and sample from it
    def sample_bivariate_normal(mu_x,mu_y,sigma_x,sigma_y,rho_xy, greedy=False):
        if greedy:
            return mu_x, mu_y
        mean = [mu_x,mu_y]
        # https://mathworld.wolfram.com/BivariateNormalDistribution.html
    # to read more about bivariate normal and it's parameters
        sigma_x *= np.sqrt(hp.temperature)
        sigma_y *= np.sqrt(hp.temperature)
        covariance = [[sigma_x * sigma_x, rho_xy * sigma_x * sigma_y],[rho_xy * sigma_x * sigma_y, sigma_y * sigma_y]]
        x = np.random.multivariate_normal(mean, covariance, 1)
        return x[0][0], x[0][1]




In [ ]:
# Will sketch the image out using different colours for the strokes
def draw_sketch(sequence, epoch, name='_output_'):
    strokes = np.split(sequence, np.where(sequence[:,2]>0)[0]+1)
    fig = plt.figure()
    axis_1 = fig.add_subplot(1,1,1)
    
    for stroke in strokes:
        plt.plot(stoke[:,0],-stroke[:,1], color=numpy.random.rand(3,))
    
    canvas = plt.get_current_fig_manager().canvas
    canvas.draw()
    
    pil_sketch = PIL.Image.frombytes('RGB', canvas.get_width_height(), canvas.tostring_rgb())
    name = 'sketch_after_'+str(epoch)+'epochs'+name+'.jpg'
    pil_image.save(name,"JPEG")
    plt.close("all")

In [ ]:
    def reconstruction_loss(self, mask, dx, dy, p, epoch):
        # compute reconstruction loss (difference between the origin image and the reconstructed image):
        pdf = self.bivariate_normal_pdf(dx, dy)
        # loss for x and y
        LS = -torch.sum(mask*torch.log(1e-5+torch.sum(self.pi * pdf, 2)))\
            /float(Nmax*hp.batch_size)
        # loss for pen states
        LP = -torch.sum(p*torch.log(self.q))/float(Nmax*hp.batch_size)
        return LS+LP

    def kullback_leibler_loss(self):
        # compute kullbackleibler loss (difference between the distribution of latenet vector and N(0,1)):
        LKL = -0.5*torch.sum(1+self.sigma-self.mu**2-torch.exp(self.sigma))\
            /float(hp.Nz*hp.batch_size)
        if use_cuda:
            KL_min = Variable(torch.Tensor([hp.KL_min]).cuda()).detach()
        else:
            KL_min = Variable(torch.Tensor([hp.KL_min])).detach()
        return hp.wKL*self.eta_step * torch.max(LKL,KL_min)
    
    def train_temp(self, epoch):
        # compute losses:
        LKL = self.kullback_leibler_loss()
        LR = self.reconstruction_loss(mask,dx,dy,p,epoch)
        loss = LR + LKL
        # gradient step
        loss.backward()